In [5]:
import cassandra
from cassandra import ConsistencyLevel
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement, BatchStatement

from joblib import Parallel, delayed                                                                                                                                
import os
import sys
import json
import utils
import pickle

from db import *
from glob import glob
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [8]:
import os
import sys
import json
import utils
import pickle

from db import *
from glob import glob
"""
Benching Cassandra
"""

SIZE = 10000
batch_size = 250
DB = "postgres"

jobs, blobs = utils.load_pickle_data(size=SIZE)
session, cluster = connect()
session = create_tables(session)


The runtime for load_pickle_data was 0.42263317108154297 seconds.
The runtime for connect was 0.3036019802093506 seconds.
The runtime for create_tables was 0.4186227321624756 seconds.


In [8]:
start = 0
for end in range(batch_size, len(jobs)+1, batch_size):
    batch = BatchStatement(consistency_level=ConsistencyLevel.SERIAL)
    for row in jobs[start:end]:
        break

In [12]:
INSERT_STMT = """
    INSERT INTO JOB(jid, status, name, subjobs, application, backend, backend_actualCE, comment)
    VALUES          ({jid}, {status}, {name}, {subjobs}, {application}, {backend}, {backend_actualCE}, 'comment')
"""

In [22]:
len((0, 'completed', None, 1, '85c74d55153c2b535aa07b2067c9381e', 'Done', 'LCG.CNAF.it', 'comment'))

8

In [24]:
len("jid, status, name, subjobs, application, backend, backend_actualCE, comment)".split(","))

8

In [31]:
print(BATCH_STMT.replace("None", 'x'))
BATCH_STMT = BATCH_STMT.replace("None", 'x')
prep_batch = session.prepare(BATCH_STMT)

ERROR:cassandra.cluster:Error preparing query:
Traceback (most recent call last):
  File "cassandra/cluster.py", line 2965, in cassandra.cluster.Session.prepare
  File "cassandra/cluster.py", line 4729, in cassandra.cluster.ResponseFuture.result
cassandra.protocol.SyntaxException: <Error from server: code=2000 [Syntax error in CQL query] message="line 3:38 no viable alternative at input ',' (...    VALUES          (0, 'completed', [x],...)">


BEGIN BATCH
    INSERT INTO JOB(jid, status, name, subjobs, application, backend, backend_actualCE, comment)
    VALUES          (0, 'completed', x, 1, '85c74d55153c2b535aa07b2067c9381e', 'Done', 'LCG.CNAF.it', 'comment')

    INSERT INTO JOB(jid, status, name, subjobs, application, backend, backend_actualCE, comment)
    VALUES          (1, 'completed', x, 1, '85c74d55153c2b535aa07b2067c9381e', 'Done', 'LCG.CNAF.it', 'comment')

    INSERT INTO JOB(jid, status, name, subjobs, application, backend, backend_actualCE, comment)
    VALUES          (2, 'completed', x, 1, '85c74d55153c2b535aa07b2067c9381e', 'Done', 'LCG.CNAF.it', 'comment')

    INSERT INTO JOB(jid, status, name, subjobs, application, backend, backend_actualCE, comment)
    VALUES          (3, 'completed', x, 1, '85c74d55153c2b535aa07b2067c9381e', 'Done', 'LCG.CNAF.it', 'comment')

    INSERT INTO JOB(jid, status, name, subjobs, application, backend, backend_actualCE, comment)
    VALUES          (4, 'completed', x, 1, '85c

SyntaxException: <Error from server: code=2000 [Syntax error in CQL query] message="line 3:38 no viable alternative at input ',' (...    VALUES          (0, 'completed', [x],...)">

In [30]:
batch_size = 10
INSERT_STMT = """
    INSERT INTO JOB(jid, status, name, subjobs, application, backend, backend_actualCE, comment)
    VALUES          ({jid}, {status}, {name}, {subjobs}, {application}, {backend}, {backend_actualCE}, 'comment')
"""
BATCH_STMT = 'BEGIN BATCH'

for end in range(batch_size, len(jobs)+1, batch_size):
    batch = BatchStatement(consistency_level=ConsistencyLevel.SERIAL)
    BATCH_STMT = 'BEGIN BATCH'
    for row in jobs[start:end]:
        BATCH_STMT += INSERT_STMT.format(**row)
    BATCH_STMT += 'APPLY BATCH;'
    prep_batch = session.prepare(BATCH_STMT)

ERROR:cassandra.cluster:Error preparing query:
Traceback (most recent call last):
  File "cassandra/cluster.py", line 2965, in cassandra.cluster.Session.prepare
  File "cassandra/cluster.py", line 4729, in cassandra.cluster.ResponseFuture.result
cassandra.protocol.SyntaxException: <Error from server: code=2000 [Syntax error in CQL query] message="line 3:41 no viable alternative at input ',' (...    VALUES          (0, 'completed', [None],...)">


SyntaxException: <Error from server: code=2000 [Syntax error in CQL query] message="line 3:41 no viable alternative at input ',' (...    VALUES          (0, 'completed', [None],...)">

In [3]:
query = session.prepare("""
    INSERT INTO JOB(jid, status, name, subjobs, application, backend, backend_actualCE, comment)
    VALUES          (?,      ?,    ?,       ?,           ?,       ?,                ?,       ?)
""")
start = 0
with tqdm(total=int(len(jobs)/batch_size)) as progress:
    for end in range(batch_size, len(jobs)+1, batch_size):
        batch = BatchStatement(consistency_level=ConsistencyLevel.SERIAL)
        for row in jobs[start:end]:
            batch.add(query, row)

        session.execute(batch)
        progress.set_description(f"{start}-{end}")
        progress.update(1)
        start = end


9750-10000: 100%|██████████| 40/40 [00:02<00:00, 18.60it/s]


In [4]:
batch_size = 10
query = session.prepare("""
    INSERT INTO BLOB(
        jid, inputsandbox, outputsandbox, info,
        comment, time, application, backend,
        inputfiles, outputfiles, non_copyable_outputfiles,
        id, status, name, inputdir, outputdir, inputdata,
        outputdata, splitter, subjobs, master, postprocessors,
        virtualization, merger, do_auto_resubmit, metadata, been_queued,
        parallel_submit
    ) VALUES (
        ?, ?, ?, ?,
        ?, ?, ?, ?,
        ?, ?, ?,
        ?, ?, ?, ?, ?, ?,
        ?, ?, ?, ?, ?,
        ?, ?, ?, ?, ?,
        ?
    )""")
start = 0
with tqdm(total=int(len(blobs)/batch_size)) as progress:
    for end in range(batch_size, len(blobs)+1, batch_size):
        batch = BatchStatement(consistency_level=ConsistencyLevel.ANY)
        for row in blobs[start:end]:
            batch.add(query, row)

        try:
            session.execute(batch)
        except Exception as e:
            print(e)
            break
        progress.set_description(f"{start}-{end}")
        progress.update(1)
        start = end

  0%|          | 0/1000 [00:00<?, ?it/s]

Error from server: code=2200 [Invalid query] message="Batch too large"


In [5]:
batch

<BatchStatement type=LOGGED, statements=2, consistency=QUORUM>

In [52]:
rows = [*session.execute(f"""
    SELECT * from JOB
""")]
len(rows)

0

In [19]:
rows = [*session.execute(f"""
    SELECT * from BLOB
""")]
len(rows)

352

Traceback (most recent call last):
  File "cassandra/cluster.py", line 3407, in cassandra.cluster.ControlConnection._reconnect_internal
  File "cassandra/cluster.py", line 3429, in cassandra.cluster.ControlConnection._try_connect
  File "cassandra/cluster.py", line 1607, in cassandra.cluster.Cluster.connection_factory
  File "cassandra/connection.py", line 764, in cassandra.connection.Connection.factory
  File "/home/needshelp/.local/lib/python3.7/site-packages/cassandra/io/libevreactor.py", line 341, in handle_read
    buf = self._socket.recv(self.in_buffer_size)
ConnectionResetError: [Errno 104] Connection reset by peer
